# Deep Learning Arch #1
## Embedding->Dense

In [1]:
from keras.layers import Embedding, Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import os
import csv 
import sys
import re
import scipy
import numpy as np

csv.field_size_limit(sys.maxsize)

Using TensorFlow backend.


131072

In [2]:
def process_content(file_path):
    with open(file_path, "r") as file:
        lines = file.readlines()
        source_code = []
        for line in lines:
            # filter comments
            if not re.match("\s*\/\/\s*isComment", line):
                source_code.append(line)
    text = ' '.join(source_code)
    return text

In [3]:
def find_vocab_size(trainset):
    vocab = {}
    max_length = 0
    for instance in trainset:
        tokens = instance.split()

        if len(tokens) > max_length:
            max_length = len(tokens)

        for token in tokens:
            if token in vocab:
                vocab[token] += 1
            else:
                vocab[token] = 0

    return vocab, max_length

In [4]:
df = pd.read_pickle('../../data/instances.pkl')
labels = list(set(df['target'].values))
X = []
Y = []

print("Preparing lists...")
for index, row in df.iterrows():
    X.append(process_content("../" + row["source_code"]))
    Y.append(row["target"])

Preparing lists...


In [5]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
y_binary = to_categorical(encoded_Y)

X_train, X_test, y_train, y_test = train_test_split(X, y_binary, train_size = 0.75, random_state=42)

/Users/erickmaziero/virtualenvs/refact_env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [6]:
vocab, max_length = find_vocab_size(X)

vocab_size = len(vocab) + 100
encoded_docs_train = [one_hot(d, vocab_size) for d in X_train]
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=max_length, padding='post')

encoded_docs_test = [one_hot(d, vocab_size) for d in X_test]
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')

In [12]:
model1 = Sequential()
model1.add(Embedding(vocab_size, 128, input_length=max_length))
model1.add(Flatten())
model1.add(Dense(32, activation='relu'))
model1.add(Dense(y_binary.shape[1], activation='softmax'))
# compile the model
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# summarize the model
print('################# Model 1 ###############')
print(model1.summary())

model1.fit(padded_docs_train, y_train, epochs=5, verbose=1, validation_split=0.2)
# evaluate the model
loss, accuracy = model1.evaluate(padded_docs_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

################# Model 1 ###############
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 28968, 128)        21359616  
_________________________________________________________________
flatten_2 (Flatten)          (None, 3707904)           0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                118652960 
_________________________________________________________________
dense_4 (Dense)              (None, 27)                891       
Total params: 140,013,467
Trainable params: 140,013,467
Non-trainable params: 0
_________________________________________________________________
None
Train on 10555 samples, validate on 2639 samples
Epoch 1/5
10555/10555 [==============================] - 2152s 204ms/step - loss: 7.6786 - acc: 0.5212 - val_loss: 2.3026 - val_acc: 0.8571
Epoch 2/5
10555/10555 [=

KeyboardInterrupt: 

In [ ]:
model1.save()